In [2]:
import os
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class\\research'

In [3]:
os.chdir("../") # go back in directory
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class'

In [4]:
%pwd

'c:\\Users\\parva\\OneDrive\\Desktop\\MOdel BUilding\\Chicken_Dissease\\chick_Dis_Class'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# Define a frozen dataclass to hold training configuration parameters. 
# These include directories for storing models, training data, and parameters for epochs, batch size, etc.
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

# Define a dataclass to store configuration for callbacks, including directories for TensorBoard logs and model checkpoints.
@dataclass
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath:Path


In [6]:
# Importing necessary modules for configuration management and utility functions.
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf



In [7]:

# Class for managing configuration of the model and training process by reading YAML configuration files.
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        # Reading configuration and parameter YAML files and storing their content.
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # Creating the necessary directories where artifacts (like models) will be saved.
        create_directories([self.config.artifacts_root])

    # Function to retrieve the configuration for preparing callbacks.
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        # Create directories for TensorBoard logs and model checkpoints.
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        # Return an instance of the PrepareCallbacksConfig dataclass with the necessary paths.
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    # Function to retrieve the configuration for training the model.
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        # Path to the training data (after data ingestion).
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        # Create the root directory for storing training artifacts.
        create_directories([
            Path(training.root_dir)
        ])

        # Return an instance of the TrainingConfig dataclass with the necessary paths and parameters.
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import time


In [9]:
# Class for preparing the necessary callbacks (TensorBoard and ModelCheckpoint) for training.
class PrepareCallback:

    # The constructor takes a config object, which contains configurations such as log directories and checkpoint file paths.
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    # Property to create a TensorBoard callback.
    # TensorBoard helps in visualizing metrics like loss, accuracy, etc., during training.
    @property
    def _create_tb_callbacks(self):
        # Create a timestamp for uniquely naming log directories for each run.
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        
        # Define the path where the TensorBoard logs will be saved.
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,  # Root directory for TensorBoard logs.
            f"tb_logs_at_{timestamp}",            # Sub-directory with the current timestamp.
        )
        
        # Return a TensorBoard callback, which will write logs to the specified directory.
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    # Property to create a ModelCheckpoint callback.
    # ModelCheckpoint saves the model during training, only saving the best version of the model (based on validation metrics).
    @property
    def _create_ckpt_callbacks(self):
        # Ensure that the filepath is passed as a string
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),  # Convert Path to string
            save_best_only=True
        )

    # Method to get both the TensorBoard and ModelCheckpoint callbacks.
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,  # TensorBoard callback.
            self._create_ckpt_callbacks # ModelCheckpoint callback.
        ]

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
# Class for training the model. It includes methods to load the base model, prepare data generators, and execute the training process.
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    # Method to load the base model from the updated base model path.
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    # Method to prepare data generators for training and validation.
    def train_valid_generator(self):

        # Common settings for the data generator, such as rescaling and splitting data for validation.
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        # Settings for the flow_from_directory method used by the data generator.
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Create a validation data generator.
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Generate validation data from the directory.
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Check if data augmentation is enabled, and prepare the training data generator accordingly.
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        # Generate training data from the directory.
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    # Static method to save the trained model to a specified path.
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
       model.save(str(path))

    # Method to train the model with the specified callback list (TensorBoard and ModelCheckpoint).
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Train the model using the train and validation generators, and apply the callbacks for logging and saving checkpoints.
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        # Save the trained model to the specified path.
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [12]:
# Main script that initiates configuration, prepares callbacks, and starts training.
try:
    config = ConfigurationManager()  # Load configurations.
    prepare_callbacks_config = config.get_prepare_callback_config()  # Get callback configurations.
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)  # Prepare the callback object.
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()  # Get the list of callbacks (TensorBoard and ModelCheckpoint).

    training_config = config.get_training_config()  # Get training configurations.
    training = Training(config=training_config)  # Initialize the training class with configurations.
    training.get_base_model()  # Load the pre-trained base model.
    training.train_valid_generator()  # Prepare the training and validation data generators.
    training.train(
        callback_list=callback_list  # Start training the model with the callback list.
    )

# If any exception occurs during the execution, raise it.
except Exception as e:
    raise e

[2024-09-26 12:21:29,678: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-26 12:21:29,678: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-26 12:21:29,678: INFO: common: created directory at: artifacts]
[2024-09-26 12:21:29,678: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-26 12:21:29,678: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-09-26 12:21:29,692: INFO: common: created directory at: artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
19/19 [==============================] - 19s 1s/step - loss: 13.1294 - accuracy: 0.5099 - val_loss: 13.0830 - val_accuracy: 0.6094


c:\Users\parva\anaconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [14]:
### To check the traingin tensorboard view

# in the terminal 
# tensorboard --logdir artifacts/prepar_callbacks/tensorboard_log_dir/
# should display -> serving tensorboard on localhost .... 6000/ .... click and view the graph of training